# Computación cuántica práctica
Talleres que molan. Facultad de Informática, Universidad Complutense de Madrid. 31/MAR/2022.
(c) Alejandro Pozas Kerstjens

physics [a] alexpozas.com

-------
## Introducción
El principio que subyace no sólo la computación cuántica, sino todas las [tecnologías cuánticas de la información](https://qt.eu/), es que **la información es un concepto físico**. Para ser procesada y transmitida, la información tiene que estar codificada y almacenada en sistemas físicos (en cartones perforados, en dominios magnéticos, en pulsos de luz...). Debido a esto, el qué podemos hacer con la información viene determinado por las leyes de la física que rigen el comportamiento de los sistemas en los que ésta está codificada. 

En el intento de codificar información en sistemas físicos cada vez más pequeños, comenzaron a surgir preguntas como ¿Seremos alguna vez capaces de codificar un bit en un único átomo? Dado que el comportamiento de átomos individuales viene dado por la física cuántica, ¿cómo afectarán esas nuevas leyes de la naturaleza al procesamiento de la información? ¿Existirán tareas que no podamos hacer usando sistemas que se rijan por las leyes de la física clásica? ¿Dará la mecánica cuántica ventajas o inconvenientes para tareas de comunicación, computación...?
La consideración de estas preguntas dio lugar al campo de la [teoría cuántica de la información](https://en.wikipedia.org/wiki/Quantum_information). A pesar de que el procesamiento de la información codificada en sistemas cuánticos sufre de varios inconvenientes que no aparecen cuando tratamos con sistemas clásicos (por ejemplo, [la información codificada en un sistema cuántico no se puede copiar](https://es.wikipedia.org/wiki/Teorema_de_no_clonaci%C3%B3n)), muchos han sido transformados en ventajas que permiten, por ejemplo, [protocolos criptográficos completamente seguros](https://core.ac.uk/download/pdf/82447194.pdf) (en comparación con protocolos computacionamlmente seguros como RSA).

### En este taller
En este taller vamos a hacer una introducción a la computación cuántica inspirada en la computación clásica. Veremos, poco a poco, qué características nos proporciona la física cuántica y cómo lidiar con ellas para nuestro provecho. El objetivo del taller es ver un ejemplo práctico de cómo el codificar y procesar información en sistemas cuánticos puede dar una ventaja frente al procesamiento en sistemas clásicos. Este ejemplo será la simulación del protocolo [BB84](https://en.wikipedia.org/wiki/BB84) para compartir una clave de encriptación privada. El protocolo BB84 es incondicionalmente seguro (en comparación, RSA es "solo" computacionalmente seguro), y su exploración nos ayudará a introducir las herramientas de computación cuántica que se están utilizando a día de hoy tanto en investigación como en empresas.

Utilizaremos la librería [Qiskit](http://www.qiskit.org), de código abierto y desarrollada por IBM para tener una interfaz con sus ordenadores cuánticos. Veremos cómo definir circuitos, aplicar puertas lógicas, hacer medidas, y ejecutar los programas en simuladores y en ordenadores cuánticos reales.

Para comenzar, como en cualquier programa de Python, debemos importar las librerías necesarias. Dado que estamos utilizando Colab, antes de eso, debemos instalarlas en el servidor.

In [ ]:
!pip install qiskit
!pip install pylatexenc

In [ ]:
from qiskit import Aer, IBMQ         # Simulador y acceso a los chips cuánticos
from qiskit import QuantumCircuit, ClassicalRegister    # Para crear circuitos
from qiskit import execute           # Para ejecutar circuitos
import random
import numpy as np

## Building blocks de un programa cuántico
### Circuitos y medidas
Es buen momento para recordar que, a pesar de que el campo se está moviendo rápidamente, en términos de hardware cuántico aún nos encontramos más cercanos a los ordenadores de tubos de vacío y a la programación bit a bit de los años 50 que a la era de los semiconductores. Tal y como hemos dicho, el sistema cuántico más pequeño en el que se puede codificar información es el bit cuántico o qubit. En Qiskit, los qubits son procesados en circuitos, que son los objetos básicos de la librería. Consecuentemente, para introducir información en un registro de qubits y procesar la misma, lo primero que debemos hacer es definir un `QuantumCircuit` y declarar cuántos qubits queremos que contenga. De momento, vamos a contentarnos con un solo qubit:

In [ ]:
n_qubits = 1
qubit    = QuantumCircuit(n_qubits)

Pictóricamente, cada qubit representará una línea sólida en el circuito, a las cuales se irán aplicando puertas lógicas, que normalmente aparecen en forma de cajas. Para ver la representación de un circuito particular, podemos llamar a su atributo `draw`.

In [ ]:
qubit.draw(output="mpl")

Antes de entrar a las puertas lógicas, hay que mencionar una característica particular que ocurre al procesar información en sistemas cuánticos. Al final de nuestra computación, queremos tener un resultado, idealmente en forma de una cadena de bits (clásica). Eso significa que tenemos que **realizar medidas** en los sistemas cuánticos. Como veremos más adelante, esto no es tan trivial como pueda parecer a simple vista ;).

Las medidas sobre qubits se almacenan en un registro de bits clásicos, que se especifica al generar el `QuantumCircuit`. Después, las medidas se realizan llamando a la función ``measure``, en la cual se debe especificar el qubit sobre el que se realiza la medida, y el bit en el cual se debe almacenar el resultado de dicha medida.

In [ ]:
n_bits = 1
qubit_circuit = QuantumCircuit(n_qubits, n_bits)
# Medimos el qubit número 0 y almacenamos el resultado en el bit número 0
qubit_circuit.measure(qubit=0, cbit=0)
qubit_circuit.draw(output="mpl")

Una última cosa que debemos ver antes de empezar a aplicar puertas lógicas es cómo ejecutar un circuito. Esto se hace mediante la función `execute` que, a su vez, necesita de la especificación de un *backend*. Es decir, necesitamos especificar la plataforma en la cual se ejecutará el circuito. Existen tres *backends* de gran utilidad:

- El ```statevector_simulator```. Es un simulador de estados cuánticos que utiliza álgebra lineal para computar el estado resultado de un algoritmo, aplicando la matriz unitaria correspondiente al circuito para dar lugar al resultado, que en este caso es también un estado cuántico.

- El ```qasm_simulator```. Es un simulador clásico de un ordenador cuántico real. El funcionamiento es similar al ```statevector_simulator```, pero el resultado no es el estado cuántico final, sino medidas realizadas sobre este estado. Es decir, el output son cadenas de bits que representan los resultados de las medidas que realizamos a lo largo del circuito.

- Chips cuánticos reales. Cada uno tiene su propio nombre, y veremos más adelante cómo llamar a cada uno de ellos.

Los simuladores se encuentran en el módulo ```Aer``` de ```qiskit```, mientras que para acceder a los chips reales necesitamos el módulo ```IBMQ```. Comencemos con los simuladores.

In [ ]:
statevector_simulator = Aer.get_backend("statevector_simulator")
chip_simulator        = Aer.get_backend("qasm_simulator")

In [ ]:
qubit_statevector = execute(qubit, statevector_simulator)
print(qubit_statevector.result().get_statevector())

El estado cuántico de nuestro qubit inicial es un vector de dos componentes. Estas son las *amplitudes de probabilidad* de estar en el estado correspondiente al $0$ lógico y de estar en el estado correspondiente al $1$ lógico. En general, el estado de un qubit puede describirse con un vector $|\psi\rangle=(c_0, c_1)=c_0|0\rangle + c_1|1\rangle$, donde $c_0$ y $c_1$ son números complejos que satisfacen $|c_0|^2+|c_1|^2=1$. En el caso de arriba, tenemos que $c_0=1$ y $c_1=0$, de modo que el qubit está en el estado $|\psi_{\text{inic}}\rangle = |0\rangle$. De hecho, al iniciar un circuito, *todos los qubits comienzan preparados en el estado $|0\rangle$*. Sin embargo, el hecho de que podamos tener *superposiciones* de los estados $|0\rangle$ y $|1\rangle$ es una de las características que dota a la computación cuántica de gran poder, como veremos más adelante.

Veamos ahora el resultado de la ejecución segundo circuito, en el cual habíamos introducido una medida. Este es el tipo de circuitos que se envían a ejecutar a ordenadores cuánticos, siendo el resultado una cadena de bits clásicos.

In [ ]:
qubit_circuit_sim = execute(qubit_circuit, chip_simulator)
print(qubit_circuit_sim.result().get_counts())

El resultado indica que se ha ejecutado el circuito un total de $1024$ veces, y en todas ellas el valor medido del qubit ha sido el valor $0$. El qubit estaba inicializado en el estado $|0\rangle$, así que tiene bastante sentido que al medir su estado, obtengamos el valor $0$ cada vez que lo medimos. Pero, si hacemos tanto énfasis en esto, posiblemente es porque haya algo interesante detrás ;)

### Aplicando puertas lógicas
Ya hemos visto cómo crear circuitos cuánticos, medir qubits y almacenar los resultados obtenidos, y ejecutar dichos circuitos en simuladores. A continuación veremos cómo aplicar transformaciones a los qubits, para implementar operaciones y algoritmos.

En Qiskit, las puertas lógicas se aplican mediante llamadas a funciones atributo del ``QuantumCircuit``. Empecemos por algo sencillo. La puerta lógica NOT, que cambia el estado $0$ por el $1$ y viceversa, se denomina en computación cuántica la *puerta $X$*. Veamos un simple circuito en el que se ve la aplicación de dicha puerta:

In [ ]:
x_circuit = QuantumCircuit(1, 2)    # Definimos el circuito con 1 qubit y 2 bits
x_circuit.measure(0, 0)             # Medimos el estado del qubit
x_circuit.x(qubit=0)                # Aplicamos la puerta X en el qubit 0
x_circuit.measure(0, 1)             # Medimos el estado del qubit de nuevo
x_circuit.draw(output="mpl")

In [ ]:
execute(x_circuit, chip_simulator).result().get_counts()

Ahora vemos que, todas las $1024$ veces, el resultado de la primera medida es $0$, y el resultado de la segunda medida, que hemos almacenado en el segundo bit (sí, los registros están numerados de derecha a izquierda) es siempre $1$.

Vamos ahora con la primera puerta realmente cuántica, la *puerta de Hadamard*. Esta puerta lógica crea estados en superposición, puesto que transforma el estado $|0\rangle$ en el estado $\frac{1}{\sqrt{2}}(|0\rangle+|1\rangle)$, y el estado $|1\rangle$ en el estado $\frac{1}{\sqrt{2}}(|0\rangle-|1\rangle)$. Veámoslo explícitamente:

In [ ]:
# Aplicación de la puerta de Hadamard al estado |0>
hadamard_from_zero = QuantumCircuit(1)
hadamard_from_zero.h(0)
print(execute(hadamard_from_zero, statevector_simulator).result().get_statevector())

# Aplicación de la puerta de Hadamard al estado |1>
hadamard_from_one = QuantumCircuit(1)
hadamard_from_one.x(0)
hadamard_from_one.h(0)
print(execute(hadamard_from_one, statevector_simulator).result().get_statevector())

¿Qué pasará cuando hagamos una medida en estos estados? Veámoslo

In [ ]:
hadamard_from_zero.add_register(ClassicalRegister(1))
hadamard_from_zero.measure(0, 0)
print(execute(hadamard_from_zero, chip_simulator, shots=10000).result().get_counts())

hadamard_from_one.add_register(ClassicalRegister(1))
hadamard_from_one.measure(0, 0)
print(execute(hadamard_from_one, chip_simulator, shots=10000).result().get_counts())

Parece ser que, en ambos casos, más o menos la mitad de las veces el resultado de la medida es $0$, y la otra mitad de las veces es $1$. Esto es debido a que **la física cuántica describe una naturaleza probabilista**. Cuando un qubit (o cualquier sistema cuántico) se encuentra en un estado de superposición y se realiza una medida sobre él, el resultado de dicha medida es una de las posibles opciones de la superposición, con una probabilidad asociada a la correspondiente amplitud de probabilidad. Esto es, si realizamos una medida en un qubit preparado en el estado $|\psi\rangle=c_0|0\rangle + c_1|1\rangle$, el resultado de dicha medida será $0$ con una probabilidad $|c_0|^2$, y $1$ con una probabilidad $|c_1|^2$. En el caso de las superposiciones tras la puerta de Hadamard, estas probabilidades son $\left|\pm\frac{1}{\sqrt{2}}\right|^2=\frac{1}{2}$.

Antes de ver puertas lógicas que involucren dos qubits, vamos a ver una última característica de la física cuántica

In [ ]:
def which_qubit_state(statevec):
    basis_states = ["|0>", "|1>"]
    state = ""
    for amplitude, basis_state in zip(statevec._data, basis_states):
        if abs(amplitude - 1) < 1e-8:
            state += "+" + basis_state
        elif abs(amplitude) < 1e-8:
            continue
        else:
            if str(amplitude)[0] == "-":
                state += str(round(amplitude, 4)) + basis_state
            else:
                state += "+" + str(round(amplitude, 4)) + basis_state
    return state[1:] if state[0] != "-" else state

for _ in range(10):
    program = execute(hadamard_from_zero, statevector_simulator)
    result  = program.result()
    measurement_result = list(result.get_counts())[0]
    statevector_result = result.get_statevector()
    print("El resultado de la medida fue " + measurement_result
        + " y el estado tras la medida es " + which_qubit_state(statevector_result))

Cuando realizamos una medida, el estado cambia al correspondiente al resultado de la medida. Este fenómeno se llama [colapso de la función de onda](https://es.wikipedia.org/wiki/Colapso_de_la_funci%C3%B3n_de_onda), y es un proceso puramente cuántico sin análogo en el mundo clásico. En el ámbito de la computación cuántica, lo que nos importa es saber que **realizar medidas destruye las superposiciones**, de modo que deberemos realizar mediciones únicamente cuando dichas superposiciones no existan, o en lugares donde tengamos muy controlado qué está ocurriendo, y deseemos que un colapso ocurra.

Para terminar con la introducción a puertas lógicas cuánticas, vamos a ver que se pueden realizar puertas que involucran a varios qubits. Un ejemplo muy importante es la puerta CNOT. Ésta es una puerta lógica aplicada a dos qubits, que aplica la puerta $X$ al segundo qubit (el qubit objetivo), solamente si el primer qubit (llamado qubit de control) está en el estado $|1\rangle$. Si el qubit de control está en el estado $|0\rangle$, entonces el qubit objetivo se mantiene intacto. Es decir, la puerta CNOT realiza las siguientes operaciones:

$$
\begin{align}
CNOT(|0\rangle|0\rangle)&=|0\rangle|0\rangle\\
CNOT(|0\rangle|1\rangle)&=|0\rangle|1\rangle\\
CNOT(|1\rangle|0\rangle)&=|1\rangle|1\rangle\\
CNOT(|1\rangle|1\rangle)&=|1\rangle|0\rangle
\end{align}
$$

O, en una línea, $CNOT(|x\rangle|y\rangle)=|x\rangle|y\oplus x\rangle$.

**Ejercicio:** Utilizando las puertas $X$, Hadamard y CNOT, crea un circuito que genere el estado cuántico $\frac{1}{\sqrt{2}}(|0\rangle|1\rangle-|1\rangle|0\rangle)$, y que después mida el estado de los dos qubits. Ejecuta el programa varias veces. ¿Qué observas en los resultados de las medidas? Eso que acabas de observar es el [entrelazamiento cuántico](https://es.wikipedia.org/wiki/Entrelazamiento_cu%C3%A1ntico).

A modo de conclusión de esta parte, se debe mencionar que no todo son $X$, Hadamard y CNOT. De hecho, el modelo de circuitos cuánticos es un [modelo de computación universal](https://en.wikipedia.org/wiki/Quantum_logic_gate#Universal_quantum_gates), es decir, existen conjuntos de puertas lógicas que permiten escribir cualquier computación como circuitos que únicamente utilizan puertas de uno de estos conjuntos. Las puertas implementables en Qiskit son suficientes para formar un conjunto universal. Todas las puertas que se pueden insertar en un circuito cuántico se pueden ver en la [documentación](https://qiskit.org/documentation/apidoc/circuit_library.html#standard-gates), aunque una descripción más amable de las puertas más comunes se puede encontrar [aquí](https://qiskit.org/textbook/ch-states/single-qubit-gates.html) y [aquí](https://qiskit.org/textbook/ch-gates/multiple-qubits-entangled-states.html#single-qubit-gates).

**Ejercicio**: Crea los estados cuánticos $|GHZ\rangle=\frac{1}{\sqrt{2}}\left(|000\rangle+|111\rangle\right)$ y $|W\rangle=\frac{1}{\sqrt{3}}\left(|100\rangle+|010\rangle+|001\rangle\right)$. El estado GHZ es posible de preparar solo con puertas CNOT y Hadamard. Crear el [estado W](https://en.wikipedia.org/wiki/W_state) es menos obvio, así que puedes implementar el circuito en la respuesta a [esta pregunta](https://physics.stackexchange.com/questions/311743/quantum-circuit-for-a-3-qubit-w-rangle-state). Tres apuntes: primero, la puerta que en la respuesta se llama $G(1/3)$ se puede implementar a través de una puerta $U3$ de qiskit (recuerda mirar [la colección de](https://qiskit.org/textbook/ch-states/single-qubit-gates.html)[ puertas comunes](https://qiskit.org/textbook/ch-gates/multiple-qubits-entangled-states.html#single-qubit-gates) y la [documentación](https://qiskit.org/documentation/apidoc/circuit_library.html#standard-gates)). Segundo, las puertas controladas que se usan en la respuesta se activan *cuando el qubit de control está en el estado $|0\rangle$*, mientras que las puertas normales se activan cuando el qubit de control está en el estado $|1\rangle$. Para cambiar de estado de activación basta con poner una puerta $X$ en cada qubit de control antes y después de la puerta controlada. Tercero, la última puerta, que es una CNOT con dos qubits de control, se llama [puerta de Toffoli](https://en.wikipedia.org/wiki/Toffoli_gate).

## Aplicación práctica: Simulación del protocolo BB84 de distribución de claves
A continuación vamos a analizar y simular el protocolo BB84. La situación es la siguiente: imaginemos que Alice y Bob quieren comunicarse un mensaje (como por ejemplo, el número de la tarjeta de crédito de Bob) a través de un canal inseguro, como puede ser internet. Si quieren mantener la información privada, Alice y Bob deben encriptar (y después ser capaces de desencriptar) el mensaje. La manera más sencilla es utilizando el [one-time pad](https://es.wikipedia.org/wiki/Libreta_de_un_solo_uso): si ambas partes disponen de una clave privada $k$, Bob puede encriptar su mensaje $m$ haciendo $e=m\oplus k$, enviar $e$ a Alice, y ésta hacer $e\oplus k = (m\oplus k)\oplus k=m$ para desencriptarlo. El problema reside en cómo ponerse de acuerdo para establecer $k$. ¿Cómo se pueden asegurar Alice y Bob que nadie les espió cuando estaban acordando cuál sería su clave de encriptado $k$? Este es el problema que el protocolo BB84 resuelve. Como veremos, el protocolo permite detectar intrusiones a la hora de establecer la clave, de tal manera que si se realiza una detección, se puede abortar y reanudar más adelante. La noción clave que permite que este protocolo funcione es que, tal y como hemos visto arriba, _las medidas sobre un sistema cuántico no dan información completa acerca de su estado_.

### El protocolo
---
El protocolo BB84 requiere de dos canales: un canal clásico (una línea de teléfono convencional, por el que se transmite información clásica) y otro cuántico (por ejemplo, una fibra óptica por la que se envían fotones individuales). Es importante ver que _ninguno de estos canales tiene por qué ser seguro_, aunque es necesario que el canal clásico esté autenticado (que Alice y Bob estén seguros, a pesar de que alguien pueda estar escuchándoles, que quien está al otro lado de la línea es el otro). El protocolo seguido es el siguiente:

1. Alice genera una cadena aleatoria $k^A\in\{0,1\}^n$, que será la clave a compartir.
2. Alice genera otra cadena aleatoria, $b^A\in\{0,1\}^n$, que se usa para codificar cada bit de $k^A$ en un qubit. La codificación es la siguiente:
    - Si $b^A_i=0$, codifica $k^A_i\rightarrow|k^A_i\rangle$.
    - Si $b^A_i=1$, codifica $k^A_i\rightarrow|(-1)^{k^A_i}\rangle=\frac{1}{\sqrt{2}}(|0\rangle+(-1)^{k^A_i}|1\rangle)$.
    
    (Por ejemplo, en fotones, el qubit puede ser la polarización de los mismos. Si $b^A_i=0$ el bit $k^A_i$ se codifica en la polarización vertical u horizontal, y si $b^A_i=1$ el bit $k^A_i$ se codifica en la polarización diagonal o antidiagonal).
3. Alice envía todos los qubits generados a Bob.
4. Bob genera una cadena aleatoria, $b^B\in\{0,1\}^n$, que usará para medir sus qubits:
    - Si $b^B_i=0$, medirá el qubit $i$ en la base horizontal-vertical.
    - Si $b^B_i=1$, medirá el qubit $i$ en la base diagonal-antidiagonal.
    
    Al final, Bob acaba con una cadena $k^B$.

Hasta aquí la comunicación cuántica. Ahora, hay una segunda fase, de comunicación puramente clásica sobre el canal público.

5. Alice y Bob hacen públicas sus cadenas $b^A$ y $b^B$, y se quedan con los bits de $k^A$ y $k^B$ donde $b^A_i=b^B_i$. Éstos dan lugar a la clave, $\tilde{k}$.
6. Como comprobación final, Alice y Bob hacen públicos una serie de bits de $\tilde{k}$, y comprueban si ambos tienen los mismos valores para estos bits. En caso afirmativo, el protocolo ha tenido éxito y pueden usar el resto de bits como su clave compartida $k$.


### Por qué funciona
---
El protocolo de BB84 se basa en el principio de que, en sistemas cuánticos, el resultado de una medida no es siempre el mismo, _a menos que midas el estado en la misma base en la que fue preparado_. Esto es exactamente lo que vimos anteriormente con al medir los estados $\frac{1}{\sqrt{2}}(|0\rangle\pm|1\rangle)$ en la base $\{|0\rangle,|1\rangle\}$. Una vez que la transmisión de información cuántica ha terminado (paso 3), Alice y Bob hacen públicas las bases en las que han preparado y medido los qubits para, precisamente, ver cuáles han sido preparados y medidos en la misma base, y así quedarse con los bits correspondientes.

Antes de ver por qué el protocolo BB84 es seguro ante atacantes, vamos a ponernos manos a la obra y a programar una ejecución ideal.

### Programación utilizando Qiskit
---
Vamos ahora a la parte divertida. Vamos a implementar cada ronda del protocolo de BB84 como un circuito cuántico, que después ejecutaremos en un simulador. Lo primero que debemos hacer es, precisamente, crear los circuitos necesarios.

Para cada qubit (todos originalmente en el estado $|0\rangle$), Alice aplica una serie de puertas lógicas en función de los bits $k_i^A$ y $b_i^A$ para preparar el estado necesario.

In [ ]:
def Alice_prepares(key, basis):
    # Definición de un circuito de un qubit
    qubit = QuantumCircuit(1)
    # Si k^A_i = 1, cambiamos el estado
    if key > 0:
        qubit.x(0)
    # Si b^A_i = 1, cambiamos la base
    if basis > 0:
        qubit.h(0)
    qubit.barrier() # Solo para distinguir partes al final
    return qubit

Por su parte Bob, al recibir cada qubit, lo mide de acuerdo a la base especificada por $b^B_i$.

In [ ]:
def Bob_measures(qubit, basis):
    # Añadir registro de bits clásicos al circuito
    bob_bit = ClassicalRegister(1, name='bob')
    qubit.add_register(bob_bit)
    # Si b^B_i = 1, cambiamos la base de medida
    if basis > 0:
        qubit.h(0)
    qubit.measure(0, bob_bit)
    return qubit

Ahora podemos simular el protocolo. Primero, Alice elige dos cadenas, $k^A$ y $b^A$, y codifica $k^A$ en qubits según las bases especificadas por $b^A$.

In [ ]:
kA = random.choices([0, 1], k=100)
bA = random.choices([0, 1], k=100)

Alice_circuits = [Alice_prepares(kAi, bAi) for kAi, bAi in zip(kA, bA)]

In [ ]:
Alice_circuits[random.choice(range(100))].draw(output="mpl")

Después de enviarle los qubits a Bob, éste los mide en bases elegidas aleatoriamente.

In [ ]:
bB = random.choices([0, 1], k=100)
Bob_circuits = [Bob_measures(qubit, bBi) for qubit, bBi in zip(Alice_circuits, bB)]

In [ ]:
Bob_circuits[random.choice(range(100))].draw(output="mpl")

Ahora que tenemos todos los elementos, podemos ejecutar las simulaciones para obtener $k^B$.

In [ ]:
kB = []
for circuit in Bob_circuits:
    execution = chip_simulator.run(circuit, shots=1)
    kBi = int(list(execution.result().get_counts())[0])
    kB.append(kBi)

Finalmente, viene la parte de comparación de cadenas. Para empezar, vemos que las cadenas $k^A$ y $k^B$ no son iguales:

In [ ]:
bA = np.array(bA)
bB = np.array(bB)
kA = np.array(kA)
kB = np.array(kB)

print(kA[:10], kB[:10])

Sin embargo, si en lugar de usar $k^A$ y $k^B$ directamente, Alice y Bob primero hablan por el canal clásico para ver en qué posiciones las cadenas $b^A$ y $b^B$ coinciden, entonces podrán establecer una clave común.

In [ ]:
print(all(kA[bA == bB] == kB[bA == bB]), len(kA[bA == bB]))

### ¿Qué pasa cuando hay un espía?
---
Hasta ahora hemos visto que el protocolo BB84 funciona. Es decir, que permite establecer una clave común entre Alice y Bob. Pero _¿es esa clave segura?_ Esa es la pregunta a la que responderemos ahora. Para ello, introduciremos un nuevo actor, Eve, que tendrá acceso al canal clásico, público pero autenticado, y al canal cuántico entre Alice y Bob. Es decir, Eve tiene acceso a todos los qubits antes de que lleguen a Bob, y a las cadenas $b^A$ y $b^B$ _después de que Bob haya medido los qubits_ (esto es importante para asegurar la seguridad del protocolo). 

La clave preparada por Alice está codificada en los qubits, de modo que, para obtener información sobre ésta, Eve debe medir los qubits recibidos. Tiene sentido que el proceso de medida sea el mismo que hará Bob, es decir, medir en una base elegida aleatoriamente. Y aquí viene el fenómeno cuántico que va a ayudar a garantizar la seguridad del protocolo: hemos visto anteriormente que, cuando un sistema es medido, _su estado colapsa_. En ocasiones, como cuando se usan fotones como qubits, es posible incluso que el sistema desaparezca al medirlo. Consecuentemente, para evitar ser detectada, Eve debe preparar un nuevo sistema y enviárselo a Bob (si no, Alice y Bob podrían detectar una intrusión al ver que Bob no recibe sistemas). Pongamos que Eve sigue el mismo método de preparación de qubits de Alice: dado el bit $k^E_i$ obtenido al medir el qubit $i$ en la base $b^E_i$, Eve codifica este bit en la base elegida para mandárselo a Bob.

Por supuesto, Eve podría seguir otros protocolos, como simplemente enviar el estado colapsado a Bob. Es posible demostrar que no existen estrategias mejores que la que hemos dicho arriba, pero la demostración queda fuera del objetivo del taller.

In [ ]:
def Eve_intercepts(qubit):
    basis = random.choice([0, 1])
    eve_bit = ClassicalRegister(1, name='eve')
    qubit.add_register(eve_bit)
    # Primero, Eve emula a Bob
    if basis > 0:
        qubit.h(0)
    qubit.measure(0, eve_bit)
    
    # Después de medir, tiene que preparar un nuevo qubit para
    # enviar a Bob y que no sospeche que hay un intruso.
    # La mejor opción es codificar el bit obtenido de la misma
    # manera que lo haría Alice
    qubit.reset(0)
    qubit.x(0).c_if(eve_bit, 1)
    if basis > 0:
        qubit.h(0)
    qubit.barrier() # Solo para distinguir partes al final
    return qubit

Con esta nueva función, podemos simular el nuevo protocolo, que incluye la acción de Eve.

In [ ]:
# Alice prepara
kA = random.choices([0, 1], k=100)
bA = random.choices([0, 1], k=100)

Alice_circuits = [Alice_prepares(kAi, bAi) for kAi, bAi in zip(kA, bA)]

# Eve intercepta
Eve_circuits = [Eve_intercepts(qubit) for qubit in Alice_circuits]

# Bob mide
bB = random.choices([0, 1], k=100)
Bob_circuits = [Bob_measures(qubit, bBi) for qubit, bBi in zip(Eve_circuits, bB)]

Bob_circuits[random.choice(range(100))].draw(output="mpl")

In [ ]:
kB = []
for circuit in Bob_circuits:
    execution = chip_simulator.run(circuit, shots=1)
    kBi = int(list(execution.result().get_counts())[0][-1])
    kB.append(kBi)

bA = np.array(bA)
bB = np.array(bB)
kA = np.array(kA)
kB = np.array(kB)

# Alice y Bob eligen los bits con los que quedarse
kA_kept = kA[bA == bB]
kB_kept = kB[bA == bB]

# Última comprobación: los primeros bits coinciden
print(kA_kept[:10], kB_kept[:10])

A diferencia del caso anterior, si Eve está escuchando el protocolo entonces las claves extraídas por Alice y Bob no son iguales. Por esto es importante hacer el último paso del protocolo. Al utilizar una muestra de las claves generadas para compararlas entre ellas, se puede detectar si hay casos en los cuales los resultados de medir en la misma base en que se preparó el estado da lugar a resultados diferentes. Éstos son señal de que un espía está interceptando los qubits e intentando obtener una copia de las claves. En caso de que se produzca esta detección, el protocolo se aborta y se reinicia más adelante, cuando el espía pueda haberse cansado de escuchar.

## Ejecución en ordenadores cuánticos reales
Una buena característica de `qiskit` es que acceder a ordenadores cuánticos reales es tan sencillo como utilizar los simuladores clásicos. Hay un pequeño paso extra, que es 'iniciar sesión' en IBM Q Experience. Esto se lleva a cabo a través del módulo `IBMQ`, y requiere la token asociada a [tu cuenta](https://quantum-computing.ibm.com/account).

In [ ]:
QX_TOKEN = 'YOUR_TOKEN'
IBMQ.enable_account(QX_TOKEN)

Ahora podemos ver qué ordenadores están disponibles para ejecutar programas.

In [ ]:
provider = IBMQ.get_provider(hub='ibm-q')
print("%20s" % "Name", "|", "N. qubits")
print("-------------------------------")
for backend in provider.backends():
    print("%20s" % backend.name(), "|", backend.configuration().n_qubits)

Una vez hayamos elegido uno, los pasos para ejecutar un programa son los mismos que cuando utilizamos simuladores.

In [ ]:
real_chip = provider.get_backend('ibmq_bogota')
result = execute(Bob_circuits[0], backend=real_chip, shots=1024)
print(result.result().get_counts())

Te habrás dado cuenta de que ahora estamos haciendo no una sola ejecución del algoritmo, sino $1024$. Esto es para mostrar que, en la vida real, habrá ocasiones en las que podemos obtener diferentes resultados incluso cuando no hay espías y codificamos y medimos en la misma base. Esto ocurre porque los ordenadores cuánticos actuales son aparatos muy sensibles, e incluso una fuente pequeña de ruido afecta el resultado de los cálculos. A pesar de que este es uno de los grandes problemas a los que se tiene que enfrentar la computación cuántica, existen ya soluciones teóricas universales, así como métodos experimentales para solventarlo.

**Ejercicio:** A pesar de lo que acabo de escribir, en realidad hemos ejecutado en el ordenador real el protocolo con espía. Prueba a ejecutar el protocolo sin espía, para convencerte de que el ruido sigue ahí.

## Conclusiones y más allá
En este taller hemos aprendido los instrumentos básicos en computación cuántica y con ellos hemos aprendido las propiedades fundamentales de la física cuántica: la superposición de estados, el colapso de la función de onda, y el entrelazamiento. Además, hemos visto un ejemplo de cómo estas propiedades se pueden explotar para poder establecer claves criptográficas de manera segura, pudiendo detectar posibles intromisiones por parte de espías.

Esto es solo el comienzo. La rama de las tecnologías y la computación cuántica está viviendo un momento de gran auge, motivado en gran parte por la entrada de grandes empresas tecnológicas y por la aparición de los primeros prototipos de ordenadores cuánticos accesibles para el gran público. A continuación recopilo una lista no exhaustiva de lecturas y recursos de interés:
  - Libros de texto, cursos y lecturas
    - [Quantum Computation and Quantum Information](ww.cambridge.org/9781107002173), de Michael Nielsen e Isaac Chuang. Poco más que decir que ha sido el libro de texto de referencia desde hace ya 20 años.
    - [Learn quantum computation using Qiskit](https://community.qiskit.org/textbook/). Un libro de texto desarrollado por IBM para introducir a la computación cuántica a través de su librería de programación.
    - El curso de [Quantum Machine Learning](https://courses.edx.org/courses/course-v1:University_of_TorontoX+UTQML101x+1T2019/course/) de la Universidad de Toronto. A pesar de que el curso no está activo, todo el contenido se puede encontrar en [el GitLab oficial](https://gitlab.com/qosf/qml-mooc).
    - [Quantum Computing in the NISQ era and beyond](https://quantum-journal.org/papers/q-2018-08-06-79/), de John Preskill. Un interesante artículo, de relativamente fácil lectura, acerca de los problemas a los que el campo de computación cuántica se tendrá que enfrentar en el futuro próximo.
  - Plataformas de computación abiertas
    - [IBM Q Experience](https://quantum-computing.ibm.com/). La plataforma de acceso a los ordenadores cuánticos de IBM.
    - [D-Wave Leap](https://cloud.dwavesys.com/leap/). La plataforma de acceso a las máquinas de temple cuántico de D-Wave. Estas máquinas realizan cálculos en otro modelo de computación, diferente al modelo de circuitos, lo cual tiene sus ventajas e inconvenientes.
    - [Azure Quantum](https://azure.microsoft.com/services/quantum). La plataforma de Microsoft para acceder a los ordenadores cuánticos de [IonQ](https://ionq.com/), [Honeywell](https://www.honeywell.com/en-us/company/quantum), y [Quantum Circuits Inc.](https://quantumcircuits.com/)
    - [Amazon Braket](https://aws.amazon.com/braket/). La plataforma dentro de Amazon Web Services para acceder a los ordenadores cuánticos de [D-Wave](https://www.dwavesys.com/), [IonQ](https://ionq.com/), [Rigetti](https://www.rigetti.com/), y [Oxford Quantum Circuits](https://oxfordquantumcircuits.com/)
  - Librerías de programación para plataformas específicas
    - [Qiskit](https://www.qiskit.org), para los ordenadores de IBM. Es la que hemos utilizado en este taller, y de hecho es una de las más completas, con una gran cantidad de funcionalidades y de algoritmos preconstruidos para [machine learning](https://qiskit.org/documentation/machine-learning/), [ciencias naturales](https://qiskit.org/documentation/nature/), [finanzas](https://qiskit.org/documentation/finance/) y [optimización avanzada](https://qiskit.org/documentation/optimization/).
    - [Rigetti Forest](https://github.com/rigetti/forest-tutorials), para los ordenadores de Rigetti, a los cuales se puede [acceder por invitación](https://www.rigetti.com/get-quantum).
    - [D-Wave Ocean](https://ocean.dwavesys.com/), para simular y trabajar con las máquinas de D-Wave.
    - [StrawberryFields](https://www.xanadu.ai/cloud#wrapper-strawberry), de Xanadu AI. Permite simular (y en el futuro, ejecutar en chips reales) ordenadores cuánticos basados en sistemas continuos en lugar de qubits.
    - [Cirq](https://github.com/quantumlib/Cirq), de Google. De momento solo sirve para realizar simulaciones, pero se espera que sera la manera de hacer interfaz con sus procesadores cuando éstos se hagan disponibles al público. También es necesario para poder ejecutar [Tensorflow Quantum](https://github.com/tensorflow/quantum).
  - Librerías de programación genéricas
    - [Q#](https://www.microsoft.com/en-us/quantum/development-kit). La librería de Microsoft, lo más parecido a C# que se puede ser. Viene acompañada por [un repositorio con tutoriales](https://github.com/Microsoft/QuantumKatas) y programas para problemas comunes.
    - [PennyLane](https://www.xanadu.ai/software/), de Xanadu AI. Es una librería dedicada a quantum machine learning, permitiendo interfaces entre librerías de machine learning clásico como Tensorflow o Pytorch y ordenadores cuánticos de una manera agnóstica, de tal manera que no está restringido a ninguna empresa o arquitectura en particular.